In [ ]:
from google.colab import drive
import os
import numpy as np
import nibabel as nib
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose
from tensorflow.keras.layers import concatenate, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

In [ ]:
print("="*60)
print("🔍 فحص توفر GPU...")
print("="*60)

# التحقق من GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✅ GPU متاح! عدد GPUs: {len(gpus)}")
    for gpu in gpus:
        print(f"   📍 {gpu}")
    # تفعيل memory growth لتجنب استهلاك كل الذاكرة
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✅ تم تفعيل GPU Memory Growth")
    except RuntimeError as e:
        print(f"⚠️ خطأ في تفعيل Memory Growth: {e}")
else:
    print("⚠️ GPU غير متاح! سيتم استخدام CPU")
    print("❗ للتفعيل: Runtime > Change runtime type > GPU")

# طباعة معلومات TensorFlow
print(f"\n📦 TensorFlow Version: {tf.__version__}")
print(f"🔧 Built with CUDA: {tf.test.is_built_with_cuda()}")
print("="*60 + "\n")

from google.colab import drive , auth
drive.mount('/content/drive')

🔍 فحص توفر GPU...
⚠️ GPU غير متاح! سيتم استخدام CPU
❗ للتفعيل: Runtime > Change runtime type > GPU

📦 TensorFlow Version: 2.19.0
🔧 Built with CUDA: True

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class Config:
    # المسارات - عدل هذه حسب بيئتك
    DATA_DIR = "/content/chest_ct_segmentation"  # تأكد من هذا المسار
    OUTPUT_DIR = "/content/results"

    # أبعاد الصورة
    IMAGE_HEIGHT = 256
    IMAGE_WIDTH = 256
    NUM_CLASSES = 3  # الخلفية، الرئة اليسرى، الرئة اليمنى

    # معاملات التدريب
    EPOCHS = 50  # تقليل للاختبار السريع
    BATCH_SIZE = 8
    LEARNING_RATE = 1e-4
    VALIDATION_SPLIT = 0.2

    # Early stopping
    PATIENCE_EARLY_STOP = 10
    PATIENCE_LR_REDUCE = 5
    RANDOM_SEED = 42

config = Config()
os.makedirs(config.OUTPUT_DIR, exist_ok=True)

In [ ]:
class Config:
    """تكوين الهايبر باراميترز والمسارات"""

    # قائمة المجلدات التي تحتوي على البيانات
    DATA_DIRS = [
        "/content/drive/MyDrive/Seg3Data",
        "/content/drive/MyDrive/Seg3Data/seg3Data_test",
        "/content/drive/MyDrive/Seg3Data/seg3Data_train2"
    ]

    # أسماء ملفات الـ modalities (بدون .nii أو .nii.gz)
    MODALITIES = ['T1', 'T1_IR', 'T2_FLAIR']
    LABEL_FILE = 'LabelsForTesting'

    # أبعاد البيانات
    IMAGE_HEIGHT = 240
    IMAGE_WIDTH = 240
    NUM_CLASSES = 4

    # هايبر باراميترز
    EPOCHS = 150
    BATCH_SIZE = 32
    LEARNING_RATE = 1e-4

    # إعدادات التدريب
    RANDOM_SEED = 42
    PATIENCE_EARLY_STOP = 20
    PATIENCE_LR_REDUCE = 10

    # مسار حفظ النتائج
    OUTPUT_DIR = "/content/drive/MyDrive/Seg3Data/results"

config = Config()
os.makedirs(config.OUTPUT_DIR, exist_ok=True)


class MRIDataLoader:
    """فئة لتحميل ومعالجة بيانات MRI من مجلدات متعددة"""

    def __init__(self, data_dirs, modalities, label_file):
        self.data_dirs = data_dirs
        self.modalities = modalities
        self.label_file = label_file

    def load_nifti(self, data_dir, filename):
        """تحميل ملف NIfTI من مجلد محدد"""
        possible_paths = [
            os.path.join(data_dir, filename + '.nii'),
            os.path.join(data_dir, filename + '.nii.gz'),
            os.path.join(data_dir, filename)
        ]

        filepath = None
        for path in possible_paths:
            if os.path.exists(path):
                filepath = path
                break

        if filepath is None:
            raise FileNotFoundError(f"لم يتم العثور على الملف: {filename} في {data_dir}")

        print(f"    - تحميل: {os.path.basename(filepath)}")
        nii = nib.load(filepath)
        data = nii.get_fdata()
        return data

    def normalize_volume(self, volume):
        """تطبيع البيانات (normalization)"""
        volume = volume.astype(np.float32)
        mean = np.mean(volume[volume > 0])
        std = np.std(volume[volume > 0])
        if std > 0:
            volume = (volume - mean) / std
        return volume

    def load_data_from_folder(self, data_dir):
        """تحميل البيانات من مجلد واحد"""
        print(f"\n  📂 المجلد: {os.path.basename(data_dir)}")

        # تحميل جميع الـ modalities
        volumes = {}
        for modality in self.modalities:
            volume = self.load_nifti(data_dir, modality)
            volume = self.normalize_volume(volume)
            volumes[modality] = volume

        # تحميل الـ labels
        labels = self.load_nifti(data_dir, self.label_file)
        labels = labels.astype(np.int32)

        num_slices = labels.shape[2]
        unique_labels = np.unique(labels)
        print(f"    ✓ حجم البيانات: {labels.shape}")
        print(f"    ✓ عدد الشرائح: {num_slices}")
        print(f"    ✓ الـ classes: {unique_labels}")

        return volumes, labels, num_slices

    def load_all_data(self):
        """تحميل جميع البيانات من جميع المجلدات"""
        print("\n" + "="*60)
        print("بدء تحميل البيانات من جميع المجلدات...")
        print("="*60)

        all_volumes_list = []
        all_labels_list = []
        all_slice_indices = []
        folder_info = []

        for folder_idx, data_dir in enumerate(self.data_dirs):
            try:
                volumes, labels, num_slices = self.load_data_from_folder(data_dir)

                # حفظ معلومات المجلد
                folder_info.append({
                    'folder_idx': folder_idx,
                    'path': data_dir,
                    'num_slices': num_slices,
                    'volumes': volumes,
                    'labels': labels
                })

                # إنشاء قائمة بمؤشرات الشرائح لهذا المجلد
                for slice_idx in range(num_slices):
                    all_slice_indices.append({
                        'folder_idx': folder_idx,
                        'slice_idx': slice_idx
                    })

            except Exception as e:
                print(f"    ⚠ خطأ في تحميل المجلد {data_dir}: {e}")
                continue

        print("\n" + "="*60)
        print(f"✓ تم تحميل البيانات من {len(folder_info)} مجلد")
        print(f"✓ إجمالي الشرائح: {len(all_slice_indices)}")
        print("="*60)

        return folder_info, all_slice_indices


class MultiModalityGenerator(tf.keras.utils.Sequence):
    """Generator لتوليد batches من الـ slices من مجلدات متعددة"""

    def __init__(self, folder_info, slice_indices, batch_size,
                 num_classes, modalities, image_height, image_width, shuffle=True):
        self.folder_info = folder_info
        self.slice_indices = slice_indices
        self.batch_size = batch_size
        self.num_classes = num_classes
        self.modalities = modalities
        self.image_height = image_height
        self.image_width = image_width
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        """عدد الـ batches في كل epoch"""
        return int(np.ceil(len(self.slice_indices) / self.batch_size))

    def __getitem__(self, index):
        """توليد batch واحد"""
        batch_indices = self.indices[
            index * self.batch_size:(index + 1) * self.batch_size
        ]
        batch_slices = [self.slice_indices[i] for i in batch_indices]

        X, y = self._generate_data(batch_slices)
        return X, y

    def on_epoch_end(self):
        """خلط البيانات في نهاية كل epoch"""
        self.indices = np.arange(len(self.slice_indices))
        if self.shuffle:
            np.random.shuffle(self.indices)

    def _generate_data(self, batch_slices):
        """توليد البيانات لـ batch معين"""
        num_modalities = len(self.modalities)

        X = np.zeros((len(batch_slices), self.image_height,
                      self.image_width, num_modalities), dtype=np.float32)
        y = np.zeros((len(batch_slices), self.image_height,
                      self.image_width), dtype=np.int32)

        for i, slice_info in enumerate(batch_slices):
            folder_idx = slice_info['folder_idx']
            slice_idx = slice_info['slice_idx']

            # الحصول على البيانات من المجلد المناسب
            folder_data = self.folder_info[folder_idx]
            volumes = folder_data['volumes']
            labels = folder_data['labels']

            # تجميع جميع الـ modalities
            for j, modality in enumerate(self.modalities):
                X[i, :, :, j] = volumes[modality][:, :, slice_idx]

            y[i] = labels[:, :, slice_idx]

        # تحويل الـ labels إلى one-hot encoding
        y = to_categorical(y, num_classes=self.num_classes)

        return X, y

In [ ]:
class TrainingPipeline:
    """Pipeline كامل للتدريب على مجلدات متعددة"""

    def __init__(self, config):
        self.config = config
        np.random.seed(config.RANDOM_SEED)
        tf.random.set_seed(config.RANDOM_SEED)

    def prepare_data(self):
        """تحضير البيانات من جميع المجلدات"""
        loader = MRIDataLoader(
            self.config.DATA_DIRS,
            self.config.MODALITIES,
            self.config.LABEL_FILE
        )
        folder_info, train_slices = loader.load_all_data()



        print(f"\n📊 تقسيم البيانات:")
        print(f"  - Training slices: {len(train_slices)}")
        print(f"  - Training batches: {len(train_slices) // self.config.BATCH_SIZE}")

        return folder_info, train_slices

    def create_generators(self, folder_info, train_slices):
        """إنشاء data generators"""
        train_gen = MultiModalityGenerator(
            folder_info, train_slices,
            self.config.BATCH_SIZE,
            self.config.NUM_CLASSES,
            self.config.MODALITIES,
            self.config.IMAGE_HEIGHT,
            self.config.IMAGE_WIDTH,
            shuffle=True
        )


        return train_gen





    def get_callbacks(self):
        """إعداد callbacks للتدريب"""
        from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

        callbacks = [
            ModelCheckpoint(
                os.path.join(self.config.OUTPUT_DIR, 'best_model_val_loss.h5'),
                monitor='val_loss',
                save_best_only=True,
                mode='min',
                verbose=1
            ),
            ModelCheckpoint(
                os.path.join(self.config.OUTPUT_DIR, 'best_model_val_iou.h5'),
                monitor='val_mean_io_u',
                save_best_only=True,
                mode='max',
                verbose=1
            ),
            EarlyStopping(
                monitor='val_loss',
                patience=self.config.PATIENCE_EARLY_STOP,
                verbose=1,
                restore_best_weights=True
            ),
            ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=self.config.PATIENCE_LR_REDUCE,
                verbose=1,
                min_lr=1e-7
            )
        ]
        return callbacks

    def train(self):
        """تنفيذ التدريب الكامل"""
        print("\n" + "="*60)
        print("🚀 بدء Pipeline التدريب على مجلدات متعددة")
        print("="*60)

        # 1. تحضير البيانات
        folder_info, train_slices = self.prepare_data()

        # 2. إنشاء generators
        print("\n📦 إنشاء Data Generators...")
        train_gen= self.create_generators(
            folder_info, train_slices
        )
        print("✓ تم إنشاء Generators بنجاح!")

        model = build_unet(
            config.IMAGE_HEIGHT,
            config.IMAGE_WIDTH,
            len(config.MODALITIES),
            config.NUM_CLASSES
        )

        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=config.LEARNING_RATE),
            loss='categorical_crossentropy',
            metrics=['accuracy', tf.keras.metrics.MeanIoU(num_classes=config.NUM_CLASSES)]
        )
        # 4. التدريب
        print("\n" + "="*60)
        print("🎯 بدء التدريب...")
        print("="*60)
        history = model.fit(
            train_gen,
            epochs=self.config.EPOCHS,
            callbacks=self.get_callbacks(),
            verbose=1
        )

        print("\n" + "="*60)
        print("✅ تم الانتهاء من التدريب بنجاح!")
        print("="*60)

        return model, history

## training the model :

In [ ]:
print("\n" + "="*60)
print("🧠 MRI Multi-Folder Segmentation Pipeline")
print("="*60)
print(f"\n📁 عدد المجلدات: {len(config.DATA_DIRS)}")
for i, folder in enumerate(config.DATA_DIRS, 1):
    print(f"  {i}. {folder}")
print(f"\n📊 Modalities: {config.MODALITIES}")
print(f"🎯 Classes: {config.NUM_CLASSES}")
print(f"📈 Epochs: {config.EPOCHS}")
print(f"📦 Batch Size: {config.BATCH_SIZE}")

pipeline = TrainingPipeline(config)
model, history = pipeline.train()

# حفظ الموديل النهائي
final_model_path = os.path.join(config.OUTPUT_DIR, 'final_model.h5')
model.save(final_model_path)
print(f"\n💾 تم حفظ الموديل النهائي في: {final_model_path}")


🧠 MRI Multi-Folder Segmentation Pipeline

📁 عدد المجلدات: 3
  1. /content/drive/MyDrive/Seg3Data
  2. /content/drive/MyDrive/Seg3Data/seg3Data_test
  3. /content/drive/MyDrive/Seg3Data/seg3Data_train2

📊 Modalities: ['T1', 'T1_IR', 'T2_FLAIR']
🎯 Classes: 4
📈 Epochs: 150
📦 Batch Size: 32

🚀 بدء Pipeline التدريب على مجلدات متعددة

بدء تحميل البيانات من جميع المجلدات...

  📂 المجلد: Seg3Data
    - تحميل: T1.nii
    - تحميل: T1_IR.nii
    - تحميل: T2_FLAIR.nii
    - تحميل: LabelsForTesting.nii
    ✓ حجم البيانات: (240, 240, 48)
    ✓ عدد الشرائح: 48
    ✓ الـ classes: [0 1 2 3]

  📂 المجلد: seg3Data_test
    - تحميل: T1.nii
    - تحميل: T1_IR.nii
    - تحميل: T2_FLAIR.nii
    - تحميل: LabelsForTesting.nii
    ✓ حجم البيانات: (240, 240, 48)
    ✓ عدد الشرائح: 48
    ✓ الـ classes: [0 1 2 3]

  📂 المجلد: seg3Data_train2
    - تحميل: T1.nii
    - تحميل: T1_IR.nii
    - تحميل: T2_FLAIR.nii
    - تحميل: LabelsForTesting.nii
    ✓ حجم البيانات: (240, 240, 48)
    ✓ عدد الشرائح: 48
    ✓ الـ cla

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


## making the test data ready :

In [ ]:
class TestPipeline:
    """Pipeline للاختبار والتقييم"""

    def __init__(self, model, config):
        self.model = model
        self.config = config

    def load_test_data(self, test_data_dir="/content/drive/MyDrive/Seg3Data_test"):
        """تحميل بيانات الاختبار"""

        print(f"\n📂 تحميل بيانات الاختبار من: {test_data_dir}")

        loader = MRIDataLoader(
            test_data_dir,
            self.config.MODALITIES,
            self.config.LABEL_FILE
        )
        volumes, labels = loader.load_all_data()

        return volumes, labels

    def predict_slice(self, volumes, slice_idx):
        """التنبؤ على شريحة واحدة"""
        # تحضير البيانات
        X = np.zeros((1, self.config.IMAGE_HEIGHT,
                      self.config.IMAGE_WIDTH,
                      len(self.config.MODALITIES)), dtype=np.float32)

        for j, modality in enumerate(self.config.MODALITIES):
            X[0, :, :, j] = volumes[modality][:, :, slice_idx]

        # التنبؤ
        pred = self.model.predict(X, verbose=0)
        pred_mask = np.argmax(pred[0], axis=-1)

        return pred_mask, pred[0]

    def calculate_metrics(self, true_labels, pred_labels):
        """حساب metrics للتقييم"""
        from sklearn.metrics import accuracy_score, jaccard_score, f1_score

        # تسطيح المصفوفات
        true_flat = true_labels.flatten()
        pred_flat = pred_labels.flatten()

        # حساب Metrics
        accuracy = accuracy_score(true_flat, pred_flat)

        # IoU و Dice لكل class
        iou_per_class = []
        dice_per_class = []

        for class_id in range(self.config.NUM_CLASSES):
            iou = jaccard_score(true_flat == class_id,
                               pred_flat == class_id,
                               zero_division=0)
            dice = f1_score(true_flat == class_id,
                           pred_flat == class_id,
                           zero_division=0)
            iou_per_class.append(iou)
            dice_per_class.append(dice)

        mean_iou = np.mean(iou_per_class)
        mean_dice = np.mean(dice_per_class)

        return {
            'accuracy': accuracy,
            'mean_iou': mean_iou,
            'mean_dice': mean_dice,
            'iou_per_class': iou_per_class,
            'dice_per_class': dice_per_class
        }

    def visualize_predictions(self, volumes, labels, slice_indices,
                             save_path=None, num_cols=4):
        """رسم التنبؤات مقابل الحقيقة"""
        num_slices = len(slice_indices)
        num_rows = (num_slices + num_cols - 1) // num_cols

        fig, axes = plt.subplots(num_rows, num_cols,
                                figsize=(num_cols * 5, num_rows * 5))
        axes = axes.flatten() if num_slices > 1 else [axes]

        for idx, slice_idx in enumerate(slice_indices):
            if idx >= len(axes):
                break

            # التنبؤ
            pred_mask, pred_probs = self.predict_slice(volumes, slice_idx)
            true_mask = labels[:, :, slice_idx]

            # حساب metrics لهذه الشريحة
            slice_metrics = self.calculate_metrics(true_mask, pred_mask)

            # إنشاء صورة مركبة
            # نستخدم أول modality للعرض
            first_modality = self.config.MODALITIES[0]
            img = volumes[first_modality][:, :, slice_idx]

            # تطبيع الصورة للعرض
            img_normalized = (img - img.min()) / (img.max() - img.min() + 1e-8)

            # إنشاء overlay
            ax = axes[idx]
            ax.imshow(img_normalized, cmap='gray', alpha=0.7)

            # رسم الـ true mask والـ prediction بجانب بعض
            true_overlay = np.ma.masked_where(true_mask == 0, true_mask)
            pred_overlay = np.ma.masked_where(pred_mask == 0, pred_mask)

            # نصف الصورة true، نصفها prediction
            h = img.shape[0]

            # True mask (النصف الأيسر)
            ax.imshow(true_overlay, cmap='Set1', alpha=0.5, vmin=0,
                     vmax=self.config.NUM_CLASSES-1)
            ax.axvline(x=img.shape[1]//2, color='yellow', linewidth=2,
                      linestyle='--', label='Split')

            # Prediction mask (النصف الأيمن)
            pred_overlay_right = pred_overlay.copy()
            pred_overlay_right[:, :img.shape[1]//2] = np.ma.masked
            ax.imshow(pred_overlay_right, cmap='Set1', alpha=0.5,
                     vmin=0, vmax=self.config.NUM_CLASSES-1)

            # العنوان
            ax.set_title(f'Slice {slice_idx}\n'
                        f'Acc: {slice_metrics["accuracy"]:.3f} | '
                        f'IoU: {slice_metrics["mean_iou"]:.3f} | '
                        f'Dice: {slice_metrics["mean_dice"]:.3f}\n'
                        f'Left: Ground Truth | Right: Prediction',
                        fontsize=10)
            ax.axis('off')

        # إخفاء المحاور الفارغة
        for idx in range(num_slices, len(axes)):
            axes[idx].axis('off')

        plt.tight_layout()

        if save_path:
            plt.savefig(save_path, dpi=150, bbox_inches='tight')
            print(f"✅ تم حفظ الصور في: {save_path}")

        plt.show()

    def evaluate_all_slices(self, volumes, labels, slice_indices=None):
        """تقييم جميع الشرائح"""
        if slice_indices is None:
            slice_indices = range(labels.shape[2])

        print(f"\n📊 تقييم {len(slice_indices)} شريحة...")

        all_true = []
        all_pred = []

        for slice_idx in slice_indices:
            pred_mask, _ = self.predict_slice(volumes, slice_idx)
            true_mask = labels[:, :, slice_idx]

            all_true.append(true_mask.flatten())
            all_pred.append(pred_mask.flatten())

        all_true = np.concatenate(all_true)
        all_pred = np.concatenate(all_pred)

        # حساب Metrics الإجمالية
        metrics = self.calculate_metrics(all_true, all_pred)

        print("\n" + "="*60)
        print("📈 نتائج التقييم الإجمالية:")
        print("="*60)
        print(f"Overall Accuracy: {metrics['accuracy']:.4f}")
        print(f"Mean IoU: {metrics['mean_iou']:.4f}")
        print(f"Mean Dice: {metrics['mean_dice']:.4f}")
        print(f"\nPer-Class Metrics:")
        for i in range(self.config.NUM_CLASSES):
            print(f"  Class {i}:")
            print(f"    IoU:  {metrics['iou_per_class'][i]:.4f}")
            print(f"    Dice: {metrics['dice_per_class'][i]:.4f}")
        print("="*60)

        return metrics

## 2D testing result :

In [ ]:
from tensorflow.keras.models import load_model

# تحديد المسار الصحيح لملف الموديل
model_path = "/content/drive/MyDrive/Seg3Data/results/best_model_val_acc.h5"

# تحميل الموديل
model = load_model(model_path)

print("✅ تم تحميل الموديل بنجاح!")

# إنشاء Test Pipeline


test_pipeline = TestPipeline(model, config)

print("\n💡 استخدام Test Pipeline:")
print("="*60)

# ============================
# 🧪 خلية اختبار كاملة ومستقلة - عرض منفصل
# ============================

import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, jaccard_score, f1_score
import random

print("="*60)
print("🧪 خلية الاختبار الكاملة - عرض منفصل")
print("="*60)

# ============================
# 1. الإعدادات
# ============================
# مسار بيانات الاختبار
TEST_DATA_DIR = "/content/drive/MyDrive/Seg3Data"

# مسار النموذج المحفوظ
MODEL_PATH = "/content/drive/MyDrive/Seg3Data/results/best_model_val_acc.h5"

# مسار حفظ النتائج
RESULTS_DIR = "/content/drive/MyDrive/Seg3Data/results"

# أسماء الملفات
MODALITIES = ['T1', 'T1_IR', 'T2_FLAIR']
LABEL_FILE = 'LabelsForTesting'

# إعدادات أخرى
IMAGE_HEIGHT = 240
IMAGE_WIDTH = 240
NUM_CLASSES = 4  # غيّرها حسب عدد الـ classes عندك

# عدد العينات المراد عرضها
NUM_SAMPLES = 5

print(f"\n📁 مسار بيانات الاختبار: {TEST_DATA_DIR}")
print(f"🤖 مسار النموذج: {MODEL_PATH}")

# ============================
# 2. تحميل النموذج
# ============================
print("\n" + "="*60)
print("🤖 تحميل النموذج...")
print("="*60)

try:
    model = load_model(MODEL_PATH)
    print("✅ تم تحميل النموذج بنجاح!")
except Exception as e:
    print(f"❌ خطأ في تحميل النموذج: {e}")
    raise

# ============================
# 3. دوال مساعدة
# ============================
def load_nifti_file(filepath):
    """تحميل ملف NIfTI"""
    if not os.path.exists(filepath):
        if os.path.exists(filepath.replace('.nii.gz', '.nii')):
            filepath = filepath.replace('.nii.gz', '.nii')
        elif os.path.exists(filepath.replace('.nii', '.nii.gz')):
            filepath = filepath.replace('.nii', '.nii.gz')
        else:
            raise FileNotFoundError(f"الملف غير موجود: {filepath}")

    nii = nib.load(filepath)
    data = nii.get_fdata()
    return data

def normalize_volume(volume):
    """تطبيع البيانات"""
    volume = volume.astype(np.float32)
    mean = np.mean(volume[volume > 0])
    std = np.std(volume[volume > 0])
    if std > 0:
        volume = (volume - mean) / std
    return volume

# ============================
# 4. تحميل بيانات الاختبار
# ============================
print("\n" + "="*60)
print("📂 تحميل بيانات الاختبار...")
print("="*60)

if not os.path.exists(TEST_DATA_DIR):
    print(f"❌ المجلد غير موجود: {TEST_DATA_DIR}")
    raise FileNotFoundError(f"المجلد غير موجود: {TEST_DATA_DIR}")

# تحميل جميع الـ modalities
volumes = {}
for modality in MODALITIES:
    filepath = os.path.join(TEST_DATA_DIR, modality + '.nii')
    if not os.path.exists(filepath):
        filepath = os.path.join(TEST_DATA_DIR, modality + '.nii.gz')

    print(f"   تحميل {modality}...")
    volume = load_nifti_file(filepath)
    volume = normalize_volume(volume)
    volumes[modality] = volume
    print(f"      ✅ الحجم: {volume.shape}")

# تحميل Labels
print(f"   تحميل {LABEL_FILE}...")
label_filepath = os.path.join(TEST_DATA_DIR, LABEL_FILE + '.nii')
if not os.path.exists(label_filepath):
    label_filepath = os.path.join(TEST_DATA_DIR, LABEL_FILE + '.nii.gz')

labels = load_nifti_file(label_filepath).astype(np.int32)
print(f"      ✅ الحجم: {labels.shape}")
print(f"      Classes الموجودة: {np.unique(labels)}")

num_slices = labels.shape[2]
print(f"\n✅ إجمالي عدد الشرائح: {num_slices}")

# ============================
# 5. دالة التنبؤ
# ============================
def predict_slice(model, volumes, slice_idx, modalities):
    """التنبؤ على شريحة واحدة"""
    X = np.zeros((1, IMAGE_HEIGHT, IMAGE_WIDTH, len(modalities)), dtype=np.float32)

    for j, modality in enumerate(modalities):
        X[0, :, :, j] = volumes[modality][:, :, slice_idx]

    pred = model.predict(X, verbose=0)
    pred_mask = np.argmax(pred[0], axis=-1)

    return pred_mask

# ============================
# 6. دالة حساب Metrics
# ============================
def calculate_metrics(true_labels, pred_labels, num_classes):
    """حساب metrics"""
    true_flat = true_labels.flatten()
    pred_flat = pred_labels.flatten()

    accuracy = accuracy_score(true_flat, pred_flat)

    iou_per_class = []
    dice_per_class = []

    for class_id in range(num_classes):
        iou = jaccard_score(true_flat == class_id,
                           pred_flat == class_id,
                           zero_division=0)
        dice = f1_score(true_flat == class_id,
                       pred_flat == class_id,
                       zero_division=0)
        iou_per_class.append(iou)
        dice_per_class.append(dice)

    mean_iou = np.mean(iou_per_class)
    mean_dice = np.mean(dice_per_class)

    return {
        'accuracy': accuracy,
        'mean_iou': mean_iou,
        'mean_dice': mean_dice,
        'iou_per_class': iou_per_class,
        'dice_per_class': dice_per_class
    }

# ============================
# 7. دالة رسم منفصلة - كل صورة لوحدها
# ============================
def visualize_predictions_separate(model, volumes, labels, slice_indices,
                                   modalities, num_classes, save_path=None):
    """رسم التنبؤات - كل صورة منفصلة"""
    num_samples = len(slice_indices)

    # كل صف يحتوي على: Image + Ground Truth + Prediction
    fig, axes = plt.subplots(num_samples, 3, figsize=(15, num_samples * 5))

    # إذا كان عينة واحدة فقط
    if num_samples == 1:
        axes = axes.reshape(1, -1)

    for row_idx, slice_idx in enumerate(slice_indices):
        # التنبؤ
        pred_mask = predict_slice(model, volumes, slice_idx, modalities)
        true_mask = labels[:, :, slice_idx]

        # حساب metrics
        slice_metrics = calculate_metrics(true_mask, pred_mask, num_classes)

        # الصورة الأصلية
        first_modality = modalities[0]
        img = volumes[first_modality][:, :, slice_idx]
        img_normalized = (img - img.min()) / (img.max() - img.min() + 1e-8)

        # العمود 1: الصورة الأصلية
        axes[row_idx, 0].imshow(img_normalized, cmap='gray')
        axes[row_idx, 0].set_title(f'Slice {slice_idx}\nOriginal Image ({first_modality})',
                                   fontsize=12, fontweight='bold')
        axes[row_idx, 0].axis('off')

        # العمود 2: Ground Truth
        axes[row_idx, 1].imshow(img_normalized, cmap='gray', alpha=0.3)
        true_overlay = np.ma.masked_where(true_mask == 0, true_mask)
        axes[row_idx, 1].imshow(true_overlay, cmap='Set1', alpha=0.7,
                               vmin=0, vmax=num_classes-1)
        axes[row_idx, 1].set_title('Ground Truth\n(الحقيقة)',
                                  fontsize=12, fontweight='bold', color='green')
        axes[row_idx, 1].axis('off')

        # العمود 3: Prediction
        axes[row_idx, 2].imshow(img_normalized, cmap='gray', alpha=0.3)
        pred_overlay = np.ma.masked_where(pred_mask == 0, pred_mask)
        axes[row_idx, 2].imshow(pred_overlay, cmap='Set1', alpha=0.7,
                               vmin=0, vmax=num_classes-1)
        axes[row_idx, 2].set_title(
            f'Model Prediction\n(التنبؤ)\n'
            f'Acc: {slice_metrics["accuracy"]:.3f} | '
            f'IoU: {slice_metrics["mean_iou"]:.3f} | '
            f'Dice: {slice_metrics["mean_dice"]:.3f}',
            fontsize=12, fontweight='bold', color='blue'
        )
        axes[row_idx, 2].axis('off')

    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"✅ تم حفظ الصورة: {save_path}")

    plt.show()

# ============================
# 8. اختيار عينات عشوائية من المنتصف
# ============================
print("\n" + "="*60)
print(f"🎲 اختيار {NUM_SAMPLES} عينات عشوائية من منتصف البيانات")
print("="*60)

# تحديد نطاق المنتصف (25% إلى 75%)
start_slice = int(num_slices * 0.25)
end_slice = int(num_slices * 0.75)

print(f"نطاق الاختيار: من شريحة {start_slice} إلى {end_slice}")

# اختيار عشوائي من المنتصف
random.seed(42)
middle_range = list(range(start_slice, end_slice))
selected_slices = sorted(random.sample(middle_range, min(NUM_SAMPLES, len(middle_range))))

print(f"الشرائح المختارة: {selected_slices}")

# ============================
# 9. عرض النتائج
# ============================
visualize_predictions_separate(
    model, volumes, labels, selected_slices,
    MODALITIES, NUM_CLASSES,
    save_path=os.path.join(RESULTS_DIR, 'test_separate_samples.png')
)

# ============================
# 10. تقييم على الشرائح المختارة
# ============================
print("\n" + "="*60)
print("📊 تقييم الشرائح المختارة")
print("="*60)

for slice_idx in selected_slices:
    pred_mask = predict_slice(model, volumes, slice_idx, MODALITIES)
    true_mask = labels[:, :, slice_idx]

    slice_metrics = calculate_metrics(true_mask, pred_mask, NUM_CLASSES)

    print(f"\nSlice {slice_idx}:")
    print(f"  Accuracy: {slice_metrics['accuracy']:.4f} ({slice_metrics['accuracy']*100:.2f}%)")
    print(f"  Mean IoU: {slice_metrics['mean_iou']:.4f}")
    print(f"  Mean Dice: {slice_metrics['mean_dice']:.4f}")

# ============================
# 11. تقييم شامل (اختياري)
# ============================
print("\n" + "="*60)
print("📊 هل تريد تقييم شامل على جميع الشرائح؟")
print("="*60)
print("يمكنك إلغاء التعليق على الكود التالي:")
print("""
all_true = []
all_pred = []

print("جاري التنبؤ على جميع الشرائح...")
for slice_idx in range(num_slices):
    if (slice_idx + 1) % 10 == 0:
        print(f"   معالجة {slice_idx + 1}/{num_slices}...")

    pred_mask = predict_slice(model, volumes, slice_idx, MODALITIES)
    true_mask = labels[:, :, slice_idx]

    all_true.append(true_mask.flatten())
    all_pred.append(pred_mask.flatten())

all_true = np.concatenate(all_true)
all_pred = np.concatenate(all_pred)

overall_metrics = calculate_metrics(all_true, all_pred, NUM_CLASSES)

print("\\n" + "="*60)
print("📈 النتائج الإجمالية على جميع الشرائح")
print("="*60)
print(f"Overall Accuracy: {overall_metrics['accuracy']:.4f}")
print(f"Mean IoU: {overall_metrics['mean_iou']:.4f}")
print(f"Mean Dice: {overall_metrics['mean_dice']:.4f}")
""")

print("\n" + "="*60)
print("🎉 انتهى الاختبار!")
print("="*60)
print(f"\n📁 الملف المحفوظ: test_separate_samples.png")
print(f"📂 المسار: {RESULTS_DIR}")

# ============================
# اختبار النموذج
# ============================

# 1. اختبار على نفس البيانات (للتحقق السريع)
print("🧪 بدء الاختبار...")
volumes_test, labels_test = test_pipeline.load_test_data()

# عرض 8 شرائح عشوائية
import random
random_slices = random.sample(range(labels_test.shape[2]), 8)
test_pipeline.visualize_predictions(
    volumes_test,
    labels_test,
    random_slices,
    save_path=os.path.join(config.OUTPUT_DIR, 'test_predictions.png')
)

# تقييم جميع الشرائح
metrics = test_pipeline.evaluate_all_slices(volumes_test, labels_test)

# 2. اختبار على بيانات من مجلد آخر
test_dir = "/content/drive/MyDrive/seg3Data_test"  # ضع مسار المجلد الجديد

volumes_new, labels_new = test_pipeline.load_test_data(test_dir)

# اختر شرائح معينة للعرض
slices_to_show = [10, 15, 20, 25, 30, 35, 40, 45]
test_pipeline.visualize_predictions(
    volumes_new,
    labels_new,
    slices_to_show,
    save_path=os.path.join(config.OUTPUT_DIR, 'new_test_predictions.png')
)

# تقييم كامل
metrics_new = test_pipeline.evaluate_all_slices(volumes_new, labels_new)

# **3D with VTK**

In [ ]:
# ============================
# 🧪 خلية اختبار مع عرض 3D محسّن
# ============================

import os
import numpy as np
import nibabel as nib
import plotly.graph_objects as go
from skimage import measure
from scipy import ndimage
import ipywidgets as widgets
from IPython.display import display
import os

print("="*60)
print("🧪 خلية الاختبار مع عرض 3D محسّن")
print("="*60)

# ============================
# 1. الإعدادات
# ============================
TEST_DATA_DIR = "/content/drive/MyDrive/Seg3Data/seg3Data_test"
MODEL_PATH = "/content/drive/MyDrive/Seg3Data/results/final_model.h5" # Updated path
RESULTS_DIR = "/content/drive/MyDrive/Seg3Data/results"

MODALITIES = ['T1', 'T1_IR', 'T2_FLAIR']
LABEL_FILE = 'LabelsForTesting'

IMAGE_HEIGHT = 240
IMAGE_WIDTH = 240
NUM_CLASSES = 4

# عدد الشرائح للعرض 3D
NUM_SLICES_3D = 24

print(f"\n📁 مسار بيانات الاختبار: {TEST_DATA_DIR}")
print(f"🤖 مسار النموذج: {MODEL_PATH}")

# ============================
# 2. تحميل النموذج
# ============================
print("\n" + "="*60)
print("🤖 تحميل النموذج...")
print("="*60)

model = load_model(MODEL_PATH)
print("✅ تم تحميل النموذج بنجاح!")

# ============================
# 3. دوال مساعدة
# ============================
def load_nifti_file(filepath):
    if not os.path.exists(filepath):
        if os.path.exists(filepath.replace('.nii.gz', '.nii')):
            filepath = filepath.replace('.nii.gz', '.nii')
        elif os.path.exists(filepath.replace('.nii', '.nii.gz')):
            filepath = filepath.replace('.nii', '.nii.gz')
    nii = nib.load(filepath)
    return nii.get_fdata()

def normalize_volume(volume):
    volume = volume.astype(np.float32)
    mean = np.mean(volume[volume > 0])
    std = np.std(volume[volume > 0])
    if std > 0:
        volume = (volume - mean) / std
    return volume

# ============================
# 4. تحميل بيانات الاختبار
# ============================
print("\n" + "="*60)
print("📂 تحميل بيانات الاختبار...")
print("="*60)

volumes = {}
for modality in MODALITIES:
    filepath = os.path.join(TEST_DATA_DIR, modality + '.nii')
    if not os.path.exists(filepath):
        filepath = os.path.join(TEST_DATA_DIR, modality + '.nii.gz')

    print(f"   تحميل {modality}...")
    volume = load_nifti_file(filepath)
    volume = normalize_volume(volume)
    volumes[modality] = volume

label_filepath = os.path.join(TEST_DATA_DIR, LABEL_FILE + '.nii')
if not os.path.exists(label_filepath):
    label_filepath = os.path.join(TEST_DATA_DIR, LABEL_FILE + '.nii.gz')

labels = load_nifti_file(label_filepath).astype(np.int32)
num_slices = labels.shape[2]
print(f"\n✅ إجمالي عدد الشرائح: {num_slices}")

# ============================
# 5. دالة التنبؤ
# ============================
def predict_slice(model, volumes, slice_idx, modalities):
    X = np.zeros((1, IMAGE_HEIGHT, IMAGE_WIDTH, len(modalities)), dtype=np.float32)
    for j, modality in enumerate(modalities):
        X[0, :, :, j] = volumes[modality][:, :, slice_idx]
    pred = model.predict(X, verbose=0)
    return np.argmax(pred[0], axis=-1)

# ============================
# 6. التنبؤ على مجموعة شرائح متتالية
# ============================
print("\n" + "="*60)
print(f"🔮 التنبؤ على {NUM_SLICES_3D} شريحة متتالية...")
print("="*60)

start_slice = 0
end_slice = start_slice + NUM_SLICES_3D

if end_slice > num_slices:
    end_slice = num_slices
    start_slice = end_slice - NUM_SLICES_3D

print(f"نطاق الشرائح: {start_slice} إلى {end_slice}")

# التنبؤ على جميع الشرائح المحددة
pred_volume = np.zeros((IMAGE_HEIGHT, IMAGE_WIDTH, NUM_SLICES_3D), dtype=np.int32)
true_volume = np.zeros((IMAGE_HEIGHT, IMAGE_WIDTH, NUM_SLICES_3D), dtype=np.int32)

print("جاري التنبؤ...")
for i, slice_idx in enumerate(range(start_slice, end_slice)):
    print(f"   {i+1}/{NUM_SLICES_3D}")
    pred_volume[:, :, i] = predict_slice(model, volumes, slice_idx, MODALITIES)
    true_volume[:, :, i] = labels[:, :, slice_idx]

print("✅ اكتمل التنبؤ!")

# ============================
# 7. دالة عرض 3D محسّنة بنمط VTK (Surface Rendering)
# ============================
def visualize_3d_surface(volume, title, save_path=None, colors=None):
    """عرض 3D بنمط Surface Rendering مشابه لـ VTK"""

    if colors is None:
        colors = {
            1: [1.0, 0.0, 0.0],    # أحمر
            2: [0.0, 1.0, 0.0],    # أخضر
            3: [0.0, 0.5, 1.0]     # أزرق سماوي
        }

    fig = plt.figure(figsize=(14, 12))
    ax = fig.add_subplot(111, projection='3d')

    # إزالة الخلفية الرمادية
    ax.set_facecolor('black')
    fig.patch.set_facecolor('black')

    # رسم كل class كسطح ثلاثي الأبعاد
    for class_id in range(1, NUM_CLASSES):
        print(f"   معالجة Class {class_id}...")

        # إنشاء mask لهذا الـ class
        class_mask = (volume == class_id).astype(np.float32)

        if np.any(class_mask):
            try:
                # استخدام marching cubes لإنشاء سطح ثلاثي الأبعاد
                verts, faces, normals, values = measure.marching_cubes(
                    class_mask,
                    level=0.5,
                    spacing=(1.0, 1.0, 1.0)
                )

                # إنشاء mesh collection
                mesh = Poly3DCollection(verts[faces], alpha=0.7)
                mesh.set_facecolor(colors.get(class_id, [1, 1, 1]))
                mesh.set_edgecolor('none')
                ax.add_collection3d(mesh)

                print(f"      ✓ تم رسم {len(faces)} مضلع")

            except Exception as e:
                print(f"      ⚠ خطأ في معالجة Class {class_id}: {e}")

    # ضبط المحاور
    ax.set_xlabel('X', color='white', fontsize=12)
    ax.set_ylabel('Y', color='white', fontsize=12)
    ax.set_zlabel('Z (Slices)', color='white', fontsize=12)
    ax.set_title(title, fontsize=16, fontweight='bold', color='white', pad=20)

    # ضبط حدود المحاور
    ax.set_xlim([0, volume.shape[1]])
    ax.set_ylim([0, volume.shape[0]])
    ax.set_zlim([0, volume.shape[2]])

    # تخصيص ألوان المحاور والشبكة
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False
    ax.xaxis.pane.set_edgecolor('gray')
    ax.yaxis.pane.set_edgecolor('gray')
    ax.zaxis.pane.set_edgecolor('gray')
    ax.grid(color='gray', alpha=0.3)

    # تغيير ألوان علامات المحاور
    ax.tick_params(colors='white')

    # ضبط زاوية العرض
    ax.view_init(elev=25, azim=45)

    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight', facecolor='black')
        print(f"✅ تم حفظ: {save_path}")

    plt.show()

# ============================
# 8. دالة عرض Volume Rendering بالألوان
# ============================
def visualize_3d_volume_slices(volume, title, save_path=None, colors=None):
    """عرض الشرائح كطبقات ثلاثية الأبعاد ملونة"""

    if colors is None:
        colors = {
            1: [1.0, 0.0, 0.0],
            2: [0.0, 1.0, 0.0],
            3: [0.0, 0.5, 1.0]
        }

    fig = plt.figure(figsize=(14, 12))
    ax = fig.add_subplot(111, projection='3d')

    ax.set_facecolor('black')
    fig.patch.set_facecolor('black')

    # رسم كل شريحة كطبقة
    step = max(1, NUM_SLICES_3D // 12)  # عرض 12 شريحة كحد أقصى

    for z_idx in range(0, volume.shape[2], step):
        slice_data = volume[:, :, z_idx]

        # إنشاء mesh grid للشريحة
        x = np.arange(0, slice_data.shape[1])
        y = np.arange(0, slice_data.shape[0])
        X, Y = np.meshgrid(x, y)
        Z = np.ones_like(X) * z_idx

        # تلوين الشريحة حسب الـ classes
        colors_array = np.zeros((slice_data.shape[0], slice_data.shape[1], 4))

        for class_id in range(1, NUM_CLASSES):
            mask = (slice_data == class_id)
            if np.any(mask):
                color = colors.get(class_id, [1, 1, 1])
                colors_array[mask] = [*color, 0.6]

        # رسم السطح
        ax.plot_surface(X, Y, Z, facecolors=colors_array,
                       shade=False, linewidth=0, antialiased=True)

    # ضبط المحاور
    ax.set_xlabel('X', color='white', fontsize=12)
    ax.set_ylabel('Y', color='white', fontsize=12)
    ax.set_zlabel('Z (Slices)', color='white', fontsize=12)
    ax.set_title(title, fontsize=16, fontweight='bold', color='white', pad=20)

    ax.set_xlim([0, volume.shape[1]])
    ax.set_ylim([0, volume.shape[0]])
    ax.set_zlim([0, volume.shape[2]])

    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False
    ax.xaxis.pane.set_edgecolor('gray')
    ax.yaxis.pane.set_edgecolor('gray')
    ax.zaxis.pane.set_edgecolor('gray')
    ax.grid(color='gray', alpha=0.3)
    ax.tick_params(colors='white')

    ax.view_init(elev=25, azim=45)

    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight', facecolor='black')
        print(f"✅ تم حفظ: {save_path}")

    plt.show()

# ============================
# 9. عرض المقارنة فقط (Ground Truth و Prediction)
# ============================
print("\n" + "="*60)
print("🎨 عرض مقارنة 3D...")
print("="*60)

# ألوان غامقة
dark_colors = {
    1: [0.7, 0.0, 0.0],    # أحمر غامق
    2: [0.0, 0.5, 0.0],    # أخضر غامق
    3: [0.0, 0.2, 0.6]     # أزرق غامق
}

fig = plt.figure(figsize=(24, 10))
fig.patch.set_facecolor('black')

# Ground Truth Surface
print("\n📊 معالجة Ground Truth...")
ax1 = fig.add_subplot(121, projection='3d')
ax1.set_facecolor('black')

for class_id in range(1, NUM_CLASSES):
    class_mask = (true_volume == class_id).astype(np.float32)
    if np.any(class_mask):
        try:
            print(f"   معالجة Class {class_id}...")
            verts, faces, _, _ = measure.marching_cubes(class_mask, level=0.5)
            mesh = Poly3DCollection(verts[faces], alpha=0.8)
            mesh.set_facecolor(dark_colors.get(class_id, [0.5, 0.5, 0.5]))
            mesh.set_edgecolor('none')
            ax1.add_collection3d(mesh)
            print(f"      ✓ تم رسم {len(faces)} مضلع")
        except Exception as e:
            print(f"      ⚠ خطأ: {e}")

ax1.set_xlim([0, true_volume.shape[1]])
ax1.set_ylim([0, true_volume.shape[0]])
ax1.set_zlim([0, true_volume.shape[2]])
ax1.set_title('Ground Truth', fontsize=16, fontweight='bold', color='white', pad=20)
ax1.set_xlabel('X', color='white', fontsize=12)
ax1.set_ylabel('Y', color='white', fontsize=12)
ax1.set_zlabel('Z (Slices)', color='white', fontsize=12)
ax1.xaxis.pane.fill = False
ax1.yaxis.pane.fill = False
ax1.zaxis.pane.fill = False
ax1.xaxis.pane.set_edgecolor('gray')
ax1.yaxis.pane.set_edgecolor('gray')
ax1.zaxis.pane.set_edgecolor('gray')
ax1.grid(color='gray', alpha=0.3)
ax1.tick_params(colors='white')
ax1.view_init(elev=25, azim=45)

# Prediction Surface
print("\n📊 معالجة Model Prediction...")
ax2 = fig.add_subplot(122, projection='3d')
ax2.set_facecolor('black')

for class_id in range(1, NUM_CLASSES):
    class_mask = (pred_volume == class_id).astype(np.float32)
    if np.any(class_mask):
        try:
            print(f"   معالجة Class {class_id}...")
            verts, faces, _, _ = measure.marching_cubes(class_mask, level=0.5)
            mesh = Poly3DCollection(verts[faces], alpha=0.8)
            mesh.set_facecolor(dark_colors.get(class_id, [0.5, 0.5, 0.5]))
            mesh.set_edgecolor('none')
            ax2.add_collection3d(mesh)
            print(f"      ✓ تم رسم {len(faces)} مضلع")
        except Exception as e:
            print(f"      ⚠ خطأ: {e}")

ax2.set_xlim([0, pred_volume.shape[1]])
ax2.set_ylim([0, pred_volume.shape[0]])
ax2.set_zlim([0, pred_volume.shape[2]])
ax2.set_title('Model Prediction', fontsize=16, fontweight='bold', color='white', pad=20)
ax2.set_xlabel('X', color='white', fontsize=12)
ax2.set_ylabel('Y', color='white', fontsize=12)
ax2.set_zlabel('Z (Slices)', color='white', fontsize=12)
ax2.xaxis.pane.fill = False
ax2.yaxis.pane.fill = False
ax2.zaxis.pane.fill = False
ax2.xaxis.pane.set_edgecolor('gray')
ax2.yaxis.pane.set_edgecolor('gray')
ax2.zaxis.pane.set_edgecolor('gray')
ax2.grid(color='gray', alpha=0.3)
ax2.tick_params(colors='white')
ax2.view_init(elev=25, azim=45)

plt.tight_layout()
save_path = os.path.join(RESULTS_DIR, '3d_comparison.png')
plt.savefig(save_path, dpi=150, bbox_inches='tight', facecolor='black')
print(f"\n✅ تم حفظ: {save_path}")
plt.show()

print("\n" + "="*60)
print("🎉 انتهى العرض 3D!")
print("="*60)
print(f"\n📁 الملف المحفوظ:")
print(f"  - 3d_comparison.png")
print(f"📂 المسار: {RESULTS_DIR}")

🧪 خلية الاختبار مع عرض 3D محسّن

📁 مسار بيانات الاختبار: /content/drive/MyDrive/Seg3Data/seg3Data_test
🤖 مسار النموذج: /content/drive/MyDrive/Seg3Data/results/final_model.h5

🤖 تحميل النموذج...


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/content/drive/MyDrive/Seg3Data/results/final_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

## 3**D predection**

In [ ]:
# ============================
# 🧪 خلية اختبار مع عرض 3D
# ============================

import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, jaccard_score, f1_score
import random

print("="*60)
print("🧪 خلية الاختبار مع عرض 3D")
print("="*60)

# ============================
# 1. الإعدادات
# ============================
TEST_DATA_DIR = "/content/drive/MyDrive/Seg3Data/seg3Data_test"
MODEL_PATH = "/content/drive/MyDrive/Seg3Data/results/final_model.h5" # Corrected path to the final model
RESULTS_DIR = "/content/drive/MyDrive/Seg3Data/results"

MODALITIES = ['T1', 'T1_IR', 'T2_FLAIR']
LABEL_FILE = 'LabelsForTesting'

IMAGE_HEIGHT = 240
IMAGE_WIDTH = 240
NUM_CLASSES = 4

# عدد الشرائح للعرض 3D
NUM_SLICES_3D = 24 # يمكنك تغييرها

print(f"\n📁 مسار بيانات الاختبار: {TEST_DATA_DIR}")
print(f"🤖 مسار النموذج: {MODEL_PATH}")

# ============================
# 2. تحميل النموذج
# ============================
print("\n" + "="*60)
print("🤖 تحميل النموذج...")
print("="*60)

model = load_model(MODEL_PATH)
print("✅ تم تحميل النموذج بنجاح!")

# ============================
# 3. دوال مساعدة
# ============================
def load_nifti_file(filepath):
    if not os.path.exists(filepath):
        if os.path.exists(filepath.replace('.nii.gz', '.nii')):
            filepath = filepath.replace('.nii.gz', '.nii')
        elif os.path.exists(filepath.replace('.nii', '.nii.gz')):
            filepath = filepath.replace('.nii', '.nii.gz')
    nii = nib.load(filepath)
    return nii.get_fdata()

def normalize_volume(volume):
    volume = volume.astype(np.float32)
    mean = np.mean(volume[volume > 0])
    std = np.std(volume[volume > 0])
    if std > 0:
        volume = (volume - mean) / std
    return volume

# ============================
# 4. تحميل بيانات الاختبار
# ============================
print("\n" + "="*60)
print("📂 تحميل بيانات الاختبار...")
print("="*60)

volumes = {}
for modality in MODALITIES:
    filepath = os.path.join(TEST_DATA_DIR, modality + '.nii')
    if not os.path.exists(filepath):
        filepath = os.path.join(TEST_DATA_DIR, modality + '.nii.gz')

    print(f"   تحميل {modality}...")
    volume = load_nifti_file(filepath)
    volume = normalize_volume(volume)
    volumes[modality] = volume

label_filepath = os.path.join(TEST_DATA_DIR, LABEL_FILE + '.nii')
if not os.path.exists(label_filepath):
    label_filepath = os.path.join(TEST_DATA_DIR, LABEL_FILE + '.nii.gz')

labels = load_nifti_file(label_filepath).astype(np.int32)
num_slices = labels.shape[2]
print(f"\n✅ إجمالي عدد الشرائح: {num_slices}")

# ============================
# 5. دالة التنبؤ
# ============================
def predict_slice(model, volumes, slice_idx, modalities):
    X = np.zeros((1, IMAGE_HEIGHT, IMAGE_WIDTH, len(modalities)), dtype=np.float32)
    for j, modality in enumerate(modalities):
        X[0, :, :, j] = volumes[modality][:, :, slice_idx]
    pred = model.predict(X, verbose=0)
    return np.argmax(pred[0], axis=-1)

# ============================
# 6. التنبؤ على مجموعة شرائح متتالية
# ============================
print("\n" + "="*60)
print(f"🔮 التنبؤ على {NUM_SLICES_3D} شريحة متتالية...")
print("="*60)

start_slice = 0
end_slice = start_slice + NUM_SLICES_3D

if end_slice > num_slices:
    end_slice = num_slices
    start_slice = end_slice - NUM_SLICES_3D

print(f"نطاق الشرائح: {start_slice} إلى {end_slice}")

# التنبؤ على جميع الشرائح المحددة
pred_volume = np.zeros((IMAGE_HEIGHT, IMAGE_WIDTH, NUM_SLICES_3D), dtype=np.int32)
true_volume = np.zeros((IMAGE_HEIGHT, IMAGE_WIDTH, NUM_SLICES_3D), dtype=np.int32)

print("جاري التنبؤ...")
for i, slice_idx in enumerate(range(start_slice, end_slice)):
    print(f"   {i+1}/{NUM_SLICES_3D}")
    pred_volume[:, :, i] = predict_slice(model, volumes, slice_idx, MODALITIES)
    true_volume[:, :, i] = labels[:, :, slice_idx]

print("✅ اكتمل التنبؤ!")

# ============================
# 7. دالة عرض 3D مع ألوان مختلفة لكل label
# ============================
def visualize_3d_segmentation(volume, title, save_path=None, colors=None):
    """عرض 3D للـ segmentation مع ألوان مختلفة"""

    # ألوان افتراضية لكل class
    if colors is None:
        colors = {
            0: [0, 0, 0, 0],          # Background - شفاف
            1: [1, 0, 0, 0.3],        # Class 1 - أحمر
            2: [0, 1, 0, 0.3],        # Class 2 - أخضر
            3: [0, 0, 1, 0.3]         # Class 3 - أزرق
        }

    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')

    # رسم كل class بلون مختلف
    for class_id in range(1, NUM_CLASSES):  # نبدأ من 1 (نتجاهل background)
        # إيجاد مواضع هذا الـ class
        class_mask = (volume == class_id)

        if np.any(class_mask):
            # الحصول على الإحداثيات
            z, y, x = np.where(class_mask)

            # رسم النقاط
            color = colors.get(class_id, [1, 1, 1, 0.3])
            ax.scatter(x, y, z,
                      c=[color[:3]],
                      alpha=color[3],
                      s=0.5,  # حجم النقطة
                      label=f'Class {class_id}')

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z (Slices)')
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.legend()

    # ضبط زاوية العرض
    ax.view_init(elev=20, azim=45)

    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"✅ تم حفظ: {save_path}")

    plt.show()

# ============================
# 8. عرض المقارنة 3D
# ============================
print("\n" + "="*60)
print("🎨 عرض النتائج بشكل 3D...")
print("="*60)

# تعريف ألوان مخصصة لكل class
custom_colors = {
    0: [0, 0, 0, 0],           # Background - شفاف
    1: [1, 0, 0, 0.4],         # Class 1 - أحمر فاتح
    2: [0, 1, 0, 0.4],         # Class 2 - أخضر فاتح
    3: [0, 0.5, 1, 0.4]        # Class 3 - أزرق سماوي
}

# عرض Ground Truth
print("\n📊 عرض Ground Truth...")
visualize_3d_segmentation(
    true_volume,
    'Ground Truth - 3D Segmentation',
    save_path=os.path.join(RESULTS_DIR, '3d_ground_truth.png'),
    colors=custom_colors
)

# عرض Prediction
print("\n📊 عرض Model Prediction...")
visualize_3d_segmentation(
    pred_volume,
    'Model Prediction - 3D Segmentation',
    save_path=os.path.join(RESULTS_DIR, '3d_prediction.png'),
    colors=custom_colors
)

# ============================
# 9. عرض جنب إلى جنب (اختياري)
# ============================
print("\n📊 عرض مقارنة جنب إلى جنب...")

fig = plt.figure(figsize=(20, 8))

# Ground Truth
ax1 = fig.add_subplot(121, projection='3d')
for class_id in range(1, NUM_CLASSES):
    class_mask = (true_volume == class_id)
    if np.any(class_mask):
        z, y, x = np.where(class_mask)
        color = custom_colors.get(class_id, [1, 1, 1, 0.3])
        ax1.scatter(x, y, z, c=[color[:3]], alpha=color[3], s=0.5, label=f'Class {class_id}')

ax1.set_title('Ground Truth', fontsize=14, fontweight='bold')
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_zlabel('Z')
ax1.legend()
ax1.view_init(elev=20, azim=45)

# Prediction
ax2 = fig.add_subplot(122, projection='3d')
for class_id in range(1, NUM_CLASSES):
    class_mask = (pred_volume == class_id)
    if np.any(class_mask):
        z, y, x = np.where(class_mask)
        color = custom_colors.get(class_id, [1, 1, 1, 0.3])
        ax2.scatter(x, y, z, c=[color[:3]], alpha=color[3], s=0.5, label=f'Class {class_id}')

ax2.set_title('Model Prediction', fontsize=14, fontweight='bold')
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.set_zlabel('Z')
ax2.legend()
ax2.view_init(elev=20, azim=45)

plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, '3d_comparison.png'), dpi=150, bbox_inches='tight')
plt.show()

print("\n" + "="*60)
print("🎉 انتهى العرض 3D!")
print("="*60)
print(f"\n📁 الملفات المحفوظة:")
print(f"  - 3d_ground_truth.png")
print(f"  - 3d_prediction.png")
print(f"  - 3d_comparison.png")
print(f"📂 المسار: {RESULTS_DIR}")

🧪 خلية الاختبار مع عرض 3D

📁 مسار بيانات الاختبار: /content/drive/MyDrive/Seg3Data/seg3Data_test
🤖 مسار النموذج: /content/drive/MyDrive/Seg3Data/results/final_model.h5

🤖 تحميل النموذج...


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/content/drive/MyDrive/Seg3Data/results/final_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
import numpy as np
import nibabel as nib
import plotly.graph_objects as go
from skimage import measure
from scipy import ndimage
import ipywidgets as widgets
from IPython.display import display
import os
from tensorflow.keras.models import load_model # Import load_model here
from mpl_toolkits.mplot3d import Axes3D # Import Axes3D here
from matplotlib import pyplot as plt # Import pyplot here
from mpl_toolkits.mplot3d.art3d import Poly3DCollection # Import Poly3DCollection

# ============================
# تثبيت المكتبات المطلوبة (قم بتشغيلها مرة واحدة)
# ============================
# !pip install plotly scikit-image ipywidgets

class Brain3DVisualizer:
    """فئة لاستخراج وعرض السطح ثلاثي الأبعاد للدماغ"""

    def __init__(self, segmentation_volume):
        """
        Parameters:
        -----------
        segmentation_volume : numpy array
            حجم الـ segmentation (3D array) مع قيم الـ classes
        """
        self.volume = segmentation_volume
        self.unique_classes = np.unique(segmentation_volume)
        self.surfaces = {}
        self.colors = {}
        self.fig = None

        # ألوان افتراضية لكل class
        self.default_colors = {
            0: 'rgba(0,0,0,0)',      # Background (شفاف)
            1: 'rgb(255,0,0)',        # أحمر
            2: 'rgb(0,255,0)',        # أخضر
            3: 'rgb(0,0,255)',        # أزرق
        }

    def extract_surface(self, class_id, step_size=2, smooth=True):
        """
        استخراج السطح ثلاثي الأبعاد لـ class معين باستخدام Marching Cubes

        Parameters:
        -----------
        class_id : int
            رقم الـ class المراد استخراج سطحه
        step_size : int
            خطوة الاستخراج (أكبر = أسرع لكن أقل دقة)
        smooth : bool
            تطبيق smoothing على الحجم قبل الاستخراج
        """
        print(f"استخراج السطح للـ class {class_id}...")

        # إنشاء binary mask للـ class
        binary_mask = (self.volume == class_id).astype(np.uint8)

        # تطبيق smoothing اختياري
        if smooth:
            binary_mask = ndimage.gaussian_filter(binary_mask.astype(float), sigma=1)

        try:
            # استخدام marching cubes لاستخراج السطح
            verts, faces, normals, values = measure.marching_cubes(
                binary_mask,
                level=0.5,
                step_size=step_size
            )

            self.surfaces[class_id] = {
                'vertices': verts,
                'faces': faces,
                'normals': normals
            }

            print(f"  ✓ تم استخراج {len(verts)} vertices و {len(faces)} faces")

        except Exception as e:
            print(f"  ✗ خطأ في استخراج السطح: {e}")
            self.surfaces[class_id] = None

    def extract_all_surfaces(self, step_size=2, smooth=True):
        """استخراج الأسطح لجميع الـ classes"""
        print("\n" + "="*60)
        print("استخراج الأسطح ثلاثية الأبعاد...")
        print("="*60)

        for class_id in self.unique_classes:
            if class_id == 0:  # تخطي الخلفية
                continue
            self.extract_surface(class_id, step_size, smooth)

        print("\n✓ تم الانتهاء من استخراج جميع الأسطح!")

    def create_mesh_trace(self, class_id, color=None, opacity=1.0, visible=True):
        """إنشاء mesh trace لـ Plotly"""

        if class_id not in self.surfaces or self.surfaces[class_id] is None:
            return None

        surface = self.surfaces[class_id]
        verts = surface['vertices']
        faces = surface['faces']

        if color is None:
            color = self.default_colors.get(class_id, 'rgb(128,128,128)')

        # إنشاء Mesh3d trace
        trace = go.Mesh3d(
            x=verts[:, 0],
            y=verts[:, 1],
            z=verts[:, 2],
            i=faces[:, 0],
            j=faces[:, 1],
            k=faces[:, 2],
            color=color,
            opacity=opacity,
            name=f'Class {class_id}',
            visible=visible,
            hoverinfo='name',
            lighting=dict(
                ambient=0.5,
                diffuse=0.8,
                specular=0.2,
                roughness=0.5
            ),
            lightposition=dict(
                x=100,
                y=200,
                z=0
            )
        )

        return trace

    def create_interactive_plot(self):
        """إنشاء الرسم ثلاثي الأبعاد التفاعلي"""

        print("\n" + "="*60)
        print("إنشاء العرض التفاعلي...")
        print("="*60)

        # إنشاء traces لجميع الـ classes
        traces = []
        for class_id in self.unique_classes:
            if class_id == 0:  # تخطي الخلفية
                continue

            trace = self.create_mesh_trace(
                class_id,
                color=self.default_colors.get(class_id),
                opacity=0.8,
                visible=True
            )

            if trace is not None:
                traces.append(trace)

        # إنشاء Figure
        self.fig = go.Figure(data=traces)

        # تحديث Layout
        self.fig.update_layout(
            title={
                'text': '🧠 Brain Segmentation - 3D Interactive Visualization',
                'x': 0.5,
                'xanchor': 'center',
                'font': {'size': 20, 'color': '#2c3e50'}
            },
            scene=dict(
                xaxis=dict(title='X', backgroundcolor="rgb(230, 230,230)"),
                yaxis=dict(title='Y', backgroundcolor="rgb(230, 230,230)"),
                zaxis=dict(title='Z', backgroundcolor="rgb(230, 230,230)"),
                aspectmode='data',
                camera=dict(
                    eye=dict(x=1.5, y=1.5, z=1.5)
                )
            ),
            width=1000,
            height=800,
            hovermode='closest',
            showlegend=True,
            legend=dict(
                x=0.02,
                y=0.98,
                bgcolor='rgba(255,255,255,0.8)',
                bordercolor='black',
                borderwidth=1
            )
        )

        print("✓ تم إنشاء العرض التفاعلي!")

        return self.fig

    def show(self):
        """عرض الرسم"""
        if self.fig is None:
            self.create_interactive_plot()
        self.fig.show()

    def create_control_widgets(self):
        """إنشاء واجهة تحكم تفاعلية"""

        print("\n" + "="*60)
        print("إنشاء عناصر التحكم التفاعلية...")
        print("="*60)

        if self.fig is None:
            self.create_interactive_plot()

        # قاموس لحفظ الـ widgets
        controls = {}

        for i, class_id in enumerate(self.unique_classes):
            if class_id == 0:  # تخطي الخلفية
                continue

            print(f"  إنشاء عناصر تحكم للـ Class {class_id}")

            # Visibility toggle
            visibility_toggle = widgets.Checkbox(
                value=True,
                description=f'Show Class {class_id}',
                style={'description_width': 'initial'}
            )

            # Opacity slider
            opacity_slider = widgets.FloatSlider(
                value=0.8,
                min=0.0,
                max=1.0,
                step=0.05,
                description=f'Opacity {class_id}:',
                style={'description_width': 'initial'}
            )

            # Color picker
            color_picker = widgets.ColorPicker(
                value=self.default_colors.get(class_id, '#808080'),
                description=f'Color {class_id}:',
                style={'description_width': 'initial'}
            )

            # دالة التحديث
            def update_trace(change, trace_idx=i):
                vis = controls[trace_idx]['visibility'].value
                opa = controls[trace_idx]['opacity'].value
                col = controls[trace_idx]['color'].value

                # تحديث الـ trace
                with self.fig.batch_update():
                    self.fig.data[trace_idx].visible = vis
                    self.fig.data[trace_idx].opacity = opa
                    self.fig.data[trace_idx].color = col

            # ربط الـ widgets بدالة التحديث
            visibility_toggle.observe(update_trace, names='value')
            opacity_slider.observe(update_trace, names='value')
            color_picker.observe(update_trace, names='value')

            # حفظ الـ widgets
            controls[i] = {
                'visibility': visibility_toggle,
                'opacity': opacity_slider,
                'color': color_picker
            }

        print("✓ تم إنشاء عناصر التحكم!")

        return controls

    def display_interactive(self):
        """عرض الواجهة التفاعلية الكاملة"""

        # إنشاء الـ controls
        controls = self.create_control_widgets()

        # ترتيب العرض
        print("\n" + "="*60)
        print("🎨 عرض الواجهة التفاعلية الكاملة")
        print("="*60)
        print("\nاستخدم عناصر التحكم أدناه لتغيير:")
        print("  • الرؤية (Show/Hide)")
        print("  • الشفافية (Opacity)")
        print("  • اللون (Color)")
        print("="*60 + "\n")

        # عرض الـ controls
        for i, control_set in controls.items():
            box = widgets.VBox([
                control_set['visibility'],
                control_set['opacity'],
                control_set['color'],
                widgets.HTML("<hr style='margin:10px 0;'>")
            ])
            display(box)

        # عرض الرسم
        display(self.fig)

    def save_html(self, filepath):
        """حفظ العرض التفاعلي كملف HTML"""
        if self.fig is None:
            self.create_interactive_plot()

        self.fig.write_html(filepath)
        print(f"\n✓ تم حفظ العرض التفاعلي في: {filepath}")


# ============================
# مثال على الاستخدام
# ============================

def visualize_segmentation_3d(segmentation_path, output_dir=None, step_size=2):
    """
    دالة رئيسية لتحميل وعرض الـ segmentation ثلاثي الأبعاد

    Parameters:
    -----------
    segmentation_path : str
        مسار ملف الـ segmentation (NIfTI)
    output_dir : str
        مسار حفظ النتائج (اختياري)
    step_size : int
        دقة استخراج السطح (أصغر = أعلى دقة لكن أبطأ)
    """

    print("\n" + "="*60)
    print("🚀 بدء عملية العرض ثلاثي الأبعاد")
    print("="*60)

    # تحميل ملف الـ segmentation
    print(f"\n📂 تحميل الملف: {segmentation_path}")
    nii = nib.load(segmentation_path)
    volume = nii.get_fdata().astype(np.int32)
    print(f"  ✓ الحجم: {volume.shape}")
    print(f"  ✓ Classes: {np.unique(volume)}")

    # إنشاء visualizer
    viz = Brain3DVisualizer(volume)

    # استخراج الأسطح
    viz.extract_all_surfaces(step_size=step_size, smooth=True)

    # عرض تفاعلي
    viz.display_interactive()

    # حفظ HTML (اختياري)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        html_path = os.path.join(output_dir, '3d_brain_visualization.html')
        viz.save_html(html_path)

    return viz


# ============================
# تشغيل على بيانات الاختبار
# ============================

# استخدم إما ملف الـ labels الأصلي أو الـ predictions
# TODO: Verify this path and update if necessary
TEST_DATA_DIR = "/content/drive/MyDrive/Seg3Data/seg3Data_test/" # <--- VERIFY THIS PATH
# TODO: Verify this file name and path and update if necessary
SEGMENTATION_FILE = os.path.join(TEST_DATA_DIR, "LabelsForTesting.nii") # <--- VERIFY THIS FILE PATH
OUTPUT_DIR = "/content/drive/MyDrive/Seg3Data/results"

# تشغيل
visualizer = visualize_segmentation_3d(
    segmentation_path=SEGMENTATION_FILE,
    output_dir=OUTPUT_DIR,
    step_size=2  # استخدم 1 لأعلى دقة (أبطأ)
)

print("\n🎉 تم الانتهاء!")


🚀 بدء عملية العرض ثلاثي الأبعاد

📂 تحميل الملف: /content/drive/MyDrive/Seg3Data/seg3Data_test/LabelsForTesting.nii


FileNotFoundError: No such file or no access: '/content/drive/MyDrive/Seg3Data/seg3Data_test/LabelsForTesting.nii'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

test_data_dir = "/content/drive/MyDrive/Seg3Data/seg3Data_test/"

print(f"Listing contents of: {test_data_dir}")
try:
    for item in os.listdir(test_data_dir):
        print(item)
except FileNotFoundError:
    print(f"Error: The directory {test_data_dir} was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Listing contents of: /content/drive/MyDrive/Seg3Data/seg3Data_test/
Error: The directory /content/drive/MyDrive/Seg3Data/seg3Data_test/ was not found.
